In [1]:
import os
import torch
import lm_eval
from tqdm import tqdm
from lm_eval.models.huggingface import HFLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from custom_neox_models import TransformerEngineGPTNeoXForCausalLM

/home/kyle/miniconda3/envs/filtering/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-14 00:51:24,663	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
model_path = "/mnt/ssd-1/kyle/checkpoints/final_hf_checkpoints/annealing_baseline_global_step11921"
model = TransformerEngineGPTNeoXForCausalLM.from_pretrained(
# model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa"
).to("cuda:0").eval()
model

Loading checkpoint shards: 100%|██████████| 6/6 [00:14<00:00,  2.41s/it]


TransformerEngineGPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (mlp): TransformerEngineGPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
          (layer_norm): LayerNorm((4096,)

## Validate Conversion

In [3]:
neox_layer_path = "/mnt/ssd-1/kyle/checkpoints/annealing_baseline/global_step11921/layer_07-model_00-model_states.pt"
neox_layer = torch.load(neox_layer_path)
neox_layer

/tmp/ipykernel_3752194/1905966943.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  neox_layer = torch.load(neox_layer_path)


OrderedDict([('input_layernorm.weight',
              tensor([0.3066, 0.3359, 0.3359,  ..., 0.3574, 0.3789, 0.3555],
                     dtype=torch.bfloat16)),
             ('input_layernorm.bias',
              tensor([ 0.0266, -0.0177,  0.0688,  ...,  0.0713,  0.0270, -0.0192],
                     dtype=torch.bfloat16)),
             ('attention.qkv.weight',
              tensor([[-2.7588e-02,  1.7212e-02,  1.1597e-02,  ...,  2.6245e-02,
                        8.6670e-03,  6.8188e-05],
                      [-2.4780e-02, -6.1646e-03,  2.1729e-02,  ...,  2.3682e-02,
                       -4.9561e-02,  9.1553e-03],
                      [ 7.1411e-03, -1.5015e-02, -4.9133e-03,  ..., -1.6113e-02,
                        8.1177e-03, -2.8534e-03],
                      ...,
                      [-1.9775e-02,  2.7466e-02, -3.0884e-02,  ...,  2.0508e-02,
                        7.6904e-03,  3.2715e-02],
                      [ 5.0049e-03, -1.0315e-02,  2.7924e-03,  ..., -3.4668e-02,
  

In [4]:
neox_layer_states_path = "/mnt/ssd-1/kyle/checkpoints/annealing_baseline/global_step11921/"
neox_model_state_files = sorted([file for file in os.listdir(neox_layer_states_path) if file.endswith("-model_states.pt")])
print(f"Loaded {len(neox_model_state_files)} model state files")
print(neox_model_state_files)
# assert len(neox_model_state_files) == 35, "Expected to load 35"

Loaded 35 model state files
['layer_00-model_00-model_states.pt', 'layer_02-model_00-model_states.pt', 'layer_03-model_00-model_states.pt', 'layer_04-model_00-model_states.pt', 'layer_05-model_00-model_states.pt', 'layer_06-model_00-model_states.pt', 'layer_07-model_00-model_states.pt', 'layer_08-model_00-model_states.pt', 'layer_09-model_00-model_states.pt', 'layer_10-model_00-model_states.pt', 'layer_11-model_00-model_states.pt', 'layer_12-model_00-model_states.pt', 'layer_13-model_00-model_states.pt', 'layer_14-model_00-model_states.pt', 'layer_15-model_00-model_states.pt', 'layer_16-model_00-model_states.pt', 'layer_17-model_00-model_states.pt', 'layer_18-model_00-model_states.pt', 'layer_19-model_00-model_states.pt', 'layer_20-model_00-model_states.pt', 'layer_21-model_00-model_states.pt', 'layer_22-model_00-model_states.pt', 'layer_23-model_00-model_states.pt', 'layer_24-model_00-model_states.pt', 'layer_25-model_00-model_states.pt', 'layer_26-model_00-model_states.pt', 'layer_27

In [5]:
module_name_map = {
    # "COLUMN_PARALLEL_LINEAR_KEYS"
    "mlp.fc1_weight": "mlp.dense_h_to_4h.weight",
    "mlp.fc1_bias": "mlp.dense_h_to_4h.bias",
    "attention.qkv.weight": "attention.query_key_value.weight",
    "attention.qkv.bias": "attention.query_key_value.bias",

    # "ROW_PARALLEL_LINEAR_KEYS"
    "attention.proj.weight": "attention.dense.weight",
    "mlp.fc2_weight": "mlp.dense_4h_to_h.weight",

    # ROW_PARALLEL_BIAS_KEYS"
    "mlp.fc2_bias": "mlp.dense_4h_to_h.bias",
    "attention.proj.bias": "attention.dense.bias",

    # "NORM_KEYS"
    "input_layernorm.weight": "input_layernorm.weight",
    "input_layernorm.bias": "input_layernorm.bias",
    "post_attention_layernorm.weight": "post_attention_layernorm.weight",
    "post_attention_layernorm.bias": "post_attention_layernorm.bias",

    # "MLP_LAYER_NORM_KEYS"
    "mlp.layer_norm_weight": "mlp.layer_norm.weight",
    "mlp.layer_norm_bias": "mlp.layer_norm.bias",

    # "FINAL_NORM_KEYS"
    "norm.weight": "weight",
    "norm.bias": "bias",
}

for layer_states_name in neox_model_state_files:
    neox_model_states = torch.load(os.path.join(neox_layer_states_path, layer_states_name), weights_only=False)
    neox_keys = [neox_key for neox_key in list(neox_model_states.keys()) if not neox_key.endswith("_extra_state")]
    neox_layer_number = int(layer_states_name.split("-model_")[0].split("layer_")[1])
    hf_layer_number = neox_layer_number - 2

    # Validate Embeddings
    if neox_layer_number == 0:
        hf_positional_embeddings= model.gpt_neox.embed_in
        neox_embeddings = neox_model_states["word_embeddings.weight"]
        assert torch.equal(hf_positional_embeddings.weight.cpu(), neox_embeddings.cpu())
        print("HF Input Embeddings Matches")
        continue

    if neox_layer_number == 1:
        continue

    # Validate Final Layer Normalization
    if neox_layer_number == 35:
        hf_final_layer_norm = model.gpt_neox.final_layer_norm
        assert torch.equal(hf_final_layer_norm.weight.cpu(), neox_model_states["norm.weight"].cpu())
        assert torch.equal(hf_final_layer_norm.bias.cpu(), neox_model_states["norm.bias"].cpu())
        print("HF Final Layer Norm Matches")
        continue

    # Validate Unembedding
    if neox_layer_number == 36:
        hf_unembedding_matrix = model.embed_out.weight
        neox_unembedding_matrix = neox_model_states["final_linear.weight"]
        assert len(neox_model_states) == 1
        assert torch.equal(hf_unembedding_matrix.cpu(), neox_unembedding_matrix.cpu())
        print("HF Unembedding Matches")
        continue

    # Validate Transformer Blocks
    hf_layer = model.gpt_neox.layers[hf_layer_number] if hf_layer_number in range(len(model.gpt_neox.layers)) else None
    if hf_layer is None:
        print(f"WARNING: Unable to HF corresponding HF layer for {layer_states_name}")
        continue

    # Ensure that all NeoX named parameters are mapped
    for neox_key in neox_keys:
        assert neox_key in module_name_map, neox_key

    # Ensure that all HF named parameters are mapped
    hf_layer_module_names = list(module[0] for module in hf_layer.named_parameters())
    assert all([name in list(module_name_map.values()) for name in hf_layer_module_names])

    # Ensure that the tensors are equal
    for neox_key in neox_keys:
        hf_key = module_name_map[neox_key]
        neox_tensor = neox_model_states[neox_key]
        hf_tensor = hf_layer
        for property_name in hf_key.split("."):
            hf_tensor = getattr(hf_tensor, property_name)

        assert torch.equal(neox_tensor.cpu(), hf_tensor.cpu())

    print(f"HF Layer {hf_layer_number} Matches")



HF Input Embeddings Matches
HF Layer 0 Matches
HF Layer 1 Matches
HF Layer 2 Matches
HF Layer 3 Matches
HF Layer 4 Matches
HF Layer 5 Matches
HF Layer 6 Matches
HF Layer 7 Matches
HF Layer 8 Matches
HF Layer 9 Matches
HF Layer 10 Matches
HF Layer 11 Matches
HF Layer 12 Matches
HF Layer 13 Matches
HF Layer 14 Matches
HF Layer 15 Matches
HF Layer 16 Matches
HF Layer 17 Matches
HF Layer 18 Matches
HF Layer 19 Matches
HF Layer 20 Matches
HF Layer 21 Matches
HF Layer 22 Matches
HF Layer 23 Matches
HF Layer 24 Matches
HF Layer 25 Matches
HF Layer 26 Matches
HF Layer 27 Matches
HF Layer 28 Matches
HF Layer 29 Matches
HF Layer 30 Matches
HF Layer 31 Matches
HF Final Layer Norm Matches
HF Unembedding Matches


## Validate Quality

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
inputs = tokenizer("COVID-19", return_tensors="pt", return_token_type_ids=False).to("cuda:0")
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


COVID-19) pandemic has had a significant impact on the global economy. The pandemic has caused a significant decline in the global economy. The pandemic has caused a significant decline in the global economy. The pandemic has caused a significant decline in the global economy. The pandemic has caused a significant decline in the global economy. The pandemic has caused a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a


In [7]:
lm_eval_model = HFLM(model)
task_manager = lm_eval.tasks.TaskManager()
qa_benchmark_results = lm_eval.simple_evaluate(
    model=lm_eval_model,
    tasks=["wmdp_bio"],
    num_fewshot=0,
    task_manager=task_manager,
    batch_size=32,
)
qa_benchmark_results


`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
[Task: wmdp_bio] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
[Task: wmdp_bio] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
Running loglikelihood requests: 100%|██████████| 5092/5092 [00:39<00:00, 129.20it/s]


{'results': {'wmdp_bio': {'alias': 'wmdp_bio',
   'acc,none': 0.3880597014925373,
   'acc_stderr,none': 0.013663445976447705}},
 'group_subtasks': {'wmdp_bio': []},
 'configs': {'wmdp_bio': {'task': 'wmdp_bio',
   'dataset_path': 'cais/wmdp',
   'dataset_name': 'wmdp-bio',
   'test_split': 'test',
   'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:',
   'doc_to_target': 'answer',
   'unsafe_code': False,
   'doc_to_choice': ['A', 'B', 'C', 'D'],
   'description': 'The following are multiple choice questions (with answers) about biology.\n\n',
   'target_delimiter': ' ',
   'fewshot_delimiter': '\n\n',
   'num_fewshot': 0,
   'metric_list': [{'metric': 'acc',
     'aggregation': 'mean',
     'higher_is_better': True}],
   'output_type': 'multiple_choice',
   'repeats': 1,
   'should_decontaminate': False,
   'metadata': {'version': 1}}},
 'versions': {'wmdp_bio': 1},
 'n-shot': {'wmdp_bio': 0},
 'higher_is_better':